# Comparing Implicit Models on LastFM Music Data
> Fitting ALS, BPR, PoissonMF, and HPFRec models on LastFM-250K music dataset

- toc: true
- badges: true
- comments: true
- categories: [Implicit, Music]
- author: "<a href='https://github.com/david-cortes'>David Cortes</a>"
- image:

## Installation

In [ ]:
# !pip install --no-use-pep517 poismf implicit hpfrec

In [37]:
import numpy as np
import pandas as pd

from scipy.sparse import coo_matrix, csr_matrix, csc_matrix

from poismf import PoisMF
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from hpfrec import HPF ### <- Bayesian version

## Data Load

### Download Alternative 1

In [ ]:
!pip install -q -U kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d neferfufi/lastfm

In [ ]:
!unzip lastfm.zip

### Download Alternative 2

In [ ]:
!wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz

In [71]:
lfm = pd.read_table('usersha1-artmbid-artname-plays.tsv',
                           sep='\t', header=None, names=['UserId','ItemId', 'Artist','Count'])
lfm.columns = ['UserId', 'ItemId', 'Artist', 'Count']
lfm.head(3)

,UserId,ItemId,Artist,Count
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897


In [79]:
lfm = lfm.drop('Artist', axis=1)
lfm = lfm.loc[lfm.Count > 0]
lfm['UserId'] = pd.Categorical(lfm.UserId).codes
lfm['ItemId'] = pd.Categorical(lfm.ItemId).codes
lfm.head(3)

,UserId,ItemId,Count
0,0,37425,2137
1,0,152039,1099
2,0,112365,897


In [80]:
lfm.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
UserId,17535654.0,179403.114983,103588.710665,0.0,89699.0,179375.0,269103.0,358867.0
ItemId,17535654.0,79521.347201,46907.281826,-1.0,39162.0,81989.0,118607.0,160111.0
Count,17535654.0,215.193173,614.481470,1.0,35.0,94.0,224.0,419157.0


### Train/Test Split

In [81]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(lfm, test_size=.3)
df_train = df_train.copy()
users_train = np.unique(df_train.UserId.to_numpy())
items_train = np.unique(df_train.ItemId.to_numpy())
df_test = df_test.loc[df_test.UserId.isin(users_train) &
                      df_test.ItemId.isin(items_train)]
df_train["UserId"] = pd.Categorical(df_train.UserId, users_train).codes
df_train["ItemId"] = pd.Categorical(df_train.ItemId, items_train).codes
df_test["UserId"] = pd.Categorical(df_test.UserId, users_train).codes
df_test["ItemId"] = pd.Categorical(df_test.ItemId, items_train).codes
users_test = np.unique(df_test.UserId.to_numpy())

print("Number of entries in training data: {:,}".format(df_train.shape[0]))
print("Number of entries in test data: {:,}".format(df_test.shape[0]))
print("Number of users in training data: {:,}".format(users_train.shape[0]))
print("Number of users in test data: {:,}".format(users_test.shape[0]))
print("Number of items in training and test data: {:,}".format(items_train.shape[0]))

Number of entries in training data: 12,274,957
Number of entries in test data: 5,245,397
Number of users in training data: 358,848
Number of users in test data: 358,698
Number of items in training and test data: 147,181


### Util function to print ranking metrics

The models fit here will be evaluated by AUC and P@5, calculated for individual users and then averaged across a random sample of 1,000 users. These metrics are calculated for each user separately, by taking the entries in the hold-out test set as a positive class, entries which are neither in the training or test sets as a negative class, and producing predictions for all the entries that were not in the training set - the idea being that models which tend to rank highest the items that the users ended up consuming are better.

In [82]:
from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed

## Note: this is a computationally inefficient implementation of the
## test metrics, not recommended to use outside of this notebook
def print_ranking_metrics(A, B, df_train, df_test, users_test,
                          nusers=1000, top_n=5, seed=1,
                          njobs=-1):
    """
    Parameters
    ----------
    A : array(m, k)
        The user-factor matrix.
    B : array(n, k)
        The item-factor matrix
    df_train : DataFrame(n_train, [user, item, value])
        The training triplets.
    df_test : DataFrame(n_test, [user, item, value])
        The hold-out triplets.
    n_user : int
        Number of users to sample.
    top_n : int
        Number of top-ranked items to calculate precision.
    seed : int
        Random seed used to select the users.
    njobs : int
        Number of jobs to run in parallel.
    """
    n_users = A.shape[0]
    n_items = B.shape[0]
    rng = np.random.default_rng(seed=seed)
    chosen_users = rng.choice(users_test, size=nusers, replace=False)
    all_train = df_train.loc[df_train.UserId.isin(chosen_users)]
    all_test = df_test.loc[df_test.UserId.isin(chosen_users)]
    
    def metrics_single_user(user):
        ypos = all_test.ItemId.loc[all_test.UserId == user].to_numpy()
        ytrain = all_train.ItemId.loc[all_train.UserId == user].to_numpy()
        yneg = np.setdiff1d(np.arange(n_items), np.r_[ypos, ytrain])
        ytest = np.r_[yneg, ypos]
        yhat = B[ytest].dot(A[user])
        auc = roc_auc_score(np.r_[np.zeros(yneg.shape[0]),
                                  np.ones(ypos.shape[0])],
                            yhat)
        topN = np.argsort(-yhat)[:top_n]
        p_at_k = np.mean(topN >= yneg.shape[0])
        p_at_k_rnd = ypos.shape[0] / ytest.shape[0] ## <- baseline
        return auc, p_at_k, p_at_k_rnd

    res_triplets = Parallel(n_jobs = njobs)\
                    (delayed(metrics_single_user)(u) \
                        for u in chosen_users)

    res_triplets = np.array(res_triplets)
    auc = np.mean(res_triplets[:,0])
    p_at_k = np.mean(res_triplets[:,1])
    p_at_k_rnd = np.mean(res_triplets[:,2])
    print("AUC: %.4f [random: %.2f]" % (auc, 0.5))
    print("P@%d: %.4f [random: %.4f]" % (top_n,
                                         p_at_k,
                                         p_at_k_rnd))

### PoisMF

**Poisson factorization**

The model is described in more detail in [Fast Non-Bayesian Poisson Factorization for Implicit-Feedback Recommendations](https://arxiv.org/abs/1811.01908).

The basic idea is to take a sparse input matrix of counts $\mathbf{X}_{m,n}$, which in this case is given by the number of times each user (row in the matrix) played each song (column in the matrix), and find an approximation as the product of two non-negative lower-dimensional latent factor matrices $\mathbf{A}_{m,k}$ and $\mathbf{B}_{n,k}$ by maximizing Poisson likelihood, i.e. fit a model:
$$
\mathbf{X} \sim \text{Poisson}(\mathbf{A} \mathbf{B}^T)
$$

Which is then used to make predictions on the missing (zero-valued) entries, with the highest-predicted items for each user being the best candidates to recommend.

The poisemf package offers different optimization methods which have different advantages in terms of speed and quality, and depending on the settings, is usually able to find good solutions in which the latent factors matrices $\mathbf{A}$ and $\mathbf{B}$ are sparse (i.e. most entries are exactly zero).
** *

In [83]:
%%time
model_fast = PoisMF(reindex=False, method="pg", use_float=False,
                    k=10, niter=10, maxupd=1, l2_reg=1e9)\
                .fit(df_train)

CPU times: user 48.6 s, sys: 99.9 ms, total: 48.7 s
Wall time: 27 s


In [84]:
print_ranking_metrics(model_fast.A, model_fast.B,
                      df_train, df_test, users_test)

AUC: 0.9538 [random: 0.50]
P@5: 0.0888 [random: 0.0001]


In [85]:
%%time
model_balanced = PoisMF(reindex=False, method="cg", use_float=False,
                        k=50, niter=30, maxupd=5, l2_reg=1e4)\
                    .fit(df_train)

CPU times: user 43min 14s, sys: 1.98 s, total: 43min 16s
Wall time: 22min 4s


In [96]:
print_ranking_metrics(model_balanced.A, model_balanced.B,
                      df_train, df_test, users_test)

AUC: 0.9823 [random: 0.50]
P@5: 0.1436 [random: 0.0001]


In [97]:
model.A[0]

array([0.06792299, 0.16954426, 0.17553033, 0.00234962, 0.05563869])

### Ranking and Prediction

In [88]:
model.topN(user = 2, n = 5, exclude = df_train.ItemId.loc[df_train.UserId==2])

array([     0,  96129, 101787,  60559, 139291])

In [89]:
model.topN_new(df_train.loc[df_train.UserId==2], n = 5, exclude = df_train.ItemId.loc[df_train.UserId==2])

array([     0,  96129,  60559, 101787, 139291])

In [90]:
model.predict(user=[3,3,3], item=[3,4,11])

array([0.00000000e+00, 2.07693311e-06, 0.00000000e+00])

### Sparse Matrix

In [91]:
## Note: package implicit takes a matrix of shape [items, users]
## Other packages take a matrix of shape [users, items]
Xcoo = coo_matrix((df_train.Count, (df_train.UserId, df_train.ItemId)))
Xcoo_T = Xcoo.T
Xcsr_T = csr_matrix(Xcoo_T)

### ALS

In [98]:
ials = AlternatingLeastSquares(factors=50, regularization=0.01,
                               dtype=np.float64, iterations=50,
                               use_gpu=False)
ials.fit(Xcsr_T)

print_ranking_metrics(ials.user_factors, ials.item_factors,
                      df_train, df_test, users_test)


AUC: 0.9782 [random: 0.50]
P@5: 0.1960 [random: 0.0001]


### BPR

In [99]:
bpr = BayesianPersonalizedRanking(factors=50, regularization=0.01,
                               dtype=np.float64, iterations=50,
                               use_gpu=False)
bpr.fit(Xcsr_T)

print_ranking_metrics(bpr.user_factors, bpr.item_factors,
                      df_train, df_test, users_test)


AUC: 0.9430 [random: 0.50]
P@5: 0.0976 [random: 0.0001]


### HPF

In [100]:
hpf = HPF(k=5, verbose=False, use_float=False).fit(Xcoo)

print_ranking_metrics(hpf.Theta, hpf.Beta,
                      df_train, df_test, users_test)

AUC: 0.9686 [random: 0.50]
P@5: 0.1086 [random: 0.0001]
